# Building Machine Learning Classifiers: Evaluate Random Forest with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

/Users/nirbhayworlikar/opt/anaconda3/envs/Practice/lib/python3.8/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/nirbhayworlikar/opt/anaconda3/envs/Practice/lib/python3.8/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/nirbhayworlikar/opt/anaconda3/envs/Practice/lib/python3.8/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/nirbhayworlikar/opt/anaconda3/envs/Practice/lib/python3.8/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names th

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,31.607065,0.447689,0.355098,0.047806,None,300,"{'max_depth': None, 'n_estimators': 300}",0.977558,0.977558,0.973046,0.968553,0.974843,0.974312,0.003350,1
8,36.522139,0.247147,0.681646,0.028033,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.979354,0.975763,0.974843,0.969452,0.970350,0.973952,0.003646,2
7,19.338419,0.204640,0.452981,0.047645,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.978456,0.976661,0.974843,0.965858,0.972147,0.973593,0.004394,3
10,20.465732,1.153994,0.433012,0.050187,None,150,"{'max_depth': None, 'n_estimators': 150}",0.980251,0.976661,0.973944,0.966757,0.970350,0.973593,0.004714,4
3,3.433844,0.239747,0.285314,0.013565,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.981149,0.974865,0.970350,0.966757,0.973046,0.973233,0.004809,5


In [4]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

/Users/nirbhayworlikar/opt/anaconda3/envs/Practice/lib/python3.8/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/nirbhayworlikar/opt/anaconda3/envs/Practice/lib/python3.8/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/nirbhayworlikar/opt/anaconda3/envs/Practice/lib/python3.8/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/nirbhayworlikar/opt/anaconda3/envs/Practice/lib/python3.8/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names th

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,17.992914,0.181625,0.464846,0.054211,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.978456,0.973968,0.973944,0.968553,0.971249,0.973234,0.003291,1
8,33.846929,0.210821,0.572244,0.047416,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977558,0.973968,0.974843,0.967655,0.970350,0.972875,0.003482,2
10,18.645946,0.930989,0.397825,0.047905,None,150,"{'max_depth': None, 'n_estimators': 150}",0.978456,0.976661,0.975741,0.967655,0.965858,0.972874,0.005103,3
11,29.785929,0.203345,0.318509,0.042896,None,300,"{'max_depth': None, 'n_estimators': 300}",0.976661,0.974865,0.973944,0.968553,0.969452,0.972695,0.003152,4
4,14.918340,0.448828,0.376146,0.026165,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.973968,0.972172,0.971249,0.964960,0.969452,0.970360,0.003069,5
